In [104]:
import scrapy
from bs4 import BeautifulSoup
from pymongo import TEXT

In [158]:
# 'img_src': response.css('div.td-post-content img::attr(src)').getall(),
import pymongo

client = pymongo.MongoClient()
db = client["news"]
# coll = db["sebenarnya_v2_test1"]
coll = db["sebenarnya_v2_proccessed1"]

q = {'category': 'COVID-19'}

In [162]:
coll.find_one()["content_text"]

[{'header': 'SEBENARNYA',
  'text': 'Jabatan Kebajikan Masyarakat (JKM) menafikan nombor talian telefon bimbit tular di media WhatsApp adalah nombor talian hotline JKM.\nSebarang urusan berhubung Bantuan COVID-19 dan sumbangan NGO, sila hubungi Bilik Gerakan JKM atau Talian Kasih yang beroperasi 24 jam.'}]

In [163]:
# list(coll.find(q))

In [139]:
x = coll.find_one()
x.keys()

dict_keys(['_id', 'date', 'category', 'url', 'title', 'content_text', 'images', 'audios', 'fact_src', 'label', 'confidence', 'search_text', 'content_html'])

In [147]:
x["date"]

datetime.datetime(2020, 4, 3, 0, 28)

In [128]:
q = coll.list_indexes()
list(q)

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'news.sebenarnya_v2_proccessed1')]),
 SON([('v', 2), ('key', SON([('_fts', 'text'), ('_ftsx', 1)])), ('name', 'content_text_text_title_text'), ('ns', 'news.sebenarnya_v2_proccessed1'), ('weights', SON([('content_text', 1), ('title', 1)])), ('default_language', 'english'), ('language_override', 'language'), ('textIndexVersion', 3)])]

In [130]:
coll.drop_indexes()

In [131]:
coll.create_index([('search_text', TEXT), ('title', TEXT)])

'search_text_text_title_text'

In [167]:
x = coll.find({"$text":{"$search": "\"Waspada Mesej Penipuan\""}})
# x = coll.find()
z = list(x)
len(z)

1

In [169]:
# z[]

In [97]:
%%time
x = coll.find_one()
list(x.keys())

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 2.4 ms


['_id',
 'date',
 'category',
 'url',
 'title',
 'content_text',
 'images',
 'audios',
 'fact_src',
 'label',
 'confidence',
 'content_html']

In [78]:
x["content_text"]

{'SEBENARNYA:': 'Jabatan Kebajikan Masyarakat (JKM) menafikan nombor talian telefon bimbit tular di media WhatsApp adalah nombor talian hotline JKM.\nSebarang urusan berhubung Bantuan COVID-19 dan sumbangan NGO, sila hubungi Bilik Gerakan JKM atau Talian Kasih yang beroperasi 24 jam.'}

In [39]:
list(coll.find_one().keys())



['_id',
 'date',
 'title',
 'content_html',
 'url',
 'label',
 'confidence',
 'category',
 'meta_full_html',
 'dep_content_text',
 'dep_img_src']

In [59]:
import datetime
dt = datetime.datetime(2020, 4,1)
category = "placeholder"
enabled_projection = ["url", "title", "category"]
proj = {key:1 for key in enabled_projection}
filter_query = {"$text": {"$search":"Palsu"}, "category": category}
x = list(coll.find(filter_query, proj).limit(2))

In [60]:
len(x)

2

In [27]:
content = coll.find_one()['content_html']
soup = BeautifulSoup(content[0], 'html.parser')
[line.strip() for line in soup.text.split('\n') if line.strip() != '']

['SEBENARNYA:',
 'Jabatan Kesihatan Negeri Johor (JKNJ) ingin merujuk kepada mesej berangkai yang tular di aplikasi WhatsApp berkaitan hebahan maklumat Kes Positif COVID-19 di Jalan Budiman, Kampung Melayu Majidee, Johor Bahru, Johor adalah tidak benar.',
 'Untuk makluman, JKN Johor ingin menafikan maklumat berkaitan dan masyarakat diminta untuk tidak menularkan sebarang berita yang tidak sahih agar tidak mencetuskan kebimbangan atau kekeliruan dikalangan masyarakat.',
 'SUMBER:',
 'Facebook Rasmi JKNJ',
 'MAINKAN PERANAN ANDA!',
 'Sekiranya anda mempunyai maklumat mengenai berita tidak ditentusah yang melibatkan kepentingan awam mahupun negara,',
 'Salurkan Kepada Kami',
 'TIDAK PASTI JANGAN KONGSI']

In [25]:
len(list(coll.find()))

30

In [17]:
# coll.drop()

{'_id': ObjectId('5e86081fde9d87570b10b145')}